<a href="https://colab.research.google.com/github/trilgar/IAD2023/blob/main/%D0%97%D0%B0%D1%80%D0%B8%D1%86%D1%8C%D0%BA%D0%B8%D0%B9_CP11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font size="6"><b>Комп'ютерний практикум 11.

<center><b> Колаборативна фільтрація </font>


<center><b><i><font size="4"> Collaborative Filtering



</b></center>



##<center>__Самостійні завдання__</center>

> Скопіювати блок самостійних завдань в окремий файл ***LastName_CP11.ipynb***

> Інсталюйте необхідні пакети бібліотек Python

In [138]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%matplotlib inline

### Завдання №1

* Завантажте архів `BX-CSV-Dump.zip` та розпакуйте дані з лінку

`url = "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"`

* Завантажте датасети `BX-Books.csv` як `df.books` та `BX-Book-Ratings.csv` як `df.ratings`, виведіть частину датасетів на екран

Використайте ці дані, щоб побудувати систему рекомендацій, яка підкаже, що користувачеві варто прочитати наступним, виходячи з його поточних книжкових уподобань.


 


In [139]:
import requests
import zipfile
import io

url = "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"
response = requests.get(url)

# Перевірка на успішне завантаження файлу
if response.status_code == 200:
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall()
    print("Архів успішно розпаковано.")
else:
    print("Помилка при завантаженні архіву.")

Архів успішно розпаковано.


In [140]:
# Завантаження датасету BX-Books.csv
books_file = "BX-Books.csv"
df_books = pd.read_csv(books_file, sep=";", error_bad_lines=False, encoding="latin-1")
print("Частину датасету BX-Books.csv:")
print(df_books.head())

# Завантаження датасету BX-Book-Ratings.csv
ratings_file = "BX-Book-Ratings.csv"
df_ratings = pd.read_csv(ratings_file, sep=";", error_bad_lines=False, encoding="latin-1")
print("Частину датасету BX-Book-Ratings.csv:")
print(df_ratings.head())

<ipython-input-140-2ab5345c9e28>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_books = pd.read_csv(books_file, sep=";", error_bad_lines=False, encoding="latin-1")
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected

Частину датасету BX-Books.csv:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0... 

In [141]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [142]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [143]:
df_ratings['User-ID'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64

In [144]:
df_ratings[df_ratings['User-ID'].isin([198711, 35859])]

,User-ID,ISBN,Book-Rating
157178,35859,0004722124,10
157179,35859,0006543936,0
157180,35859,0006547230,0
157181,35859,0007101937,0
157182,35859,0020186002,0
...,...,...,...
823616,198711,8511839102,0
823617,198711,9307166813,0
823618,198711,9590624067,0
823619,198711,9631172937,0


### Завдання №2

* Виведіть інформацію про датасет
* Залиште в датасеті лише колонки `ISBN`, `Book Title`,`Book Author` and `Publisher`.
* Перевірте, чи немає дублікатів назв книг, видаліть, якщо такі є. Для цього скористайтесь функціями `duplicated(subset='Book-Title')`  та `drop_duplicates(subset='Book-Title')`
* Об'єднайте імена та прізвища авторів, видаляючи пробіли, щоб система розрізняла авторів з однаковими іменами
* Виведіть шапку таблиці, щоб переконатись у об'єднанні



In [145]:
# Вибираємо потрібні колонки
selected_columns = ['ISBN', 'Book-Title', 'Book-Author', 'Publisher']
df_books_selected = df_books[selected_columns]

# Видаляємо дублікати назв книг
df_books_selected = df_books_selected.drop_duplicates(subset='Book-Title')

# Об'єднуємо імена та прізвища авторів, видаляючи пробіли
df_books_selected['Book-Author'] = df_books_selected['Book-Author'].str.replace(' ', '')

In [146]:
df_books_selected.head()

,ISBN,Book-Title,Book-Author,Publisher
0,0195153448,Classical Mythology,MarkP.O.Morford,Oxford University Press
1,0002005018,Clara Callan,RichardBruceWright,HarperFlamingo Canada
2,0060973129,Decision in Normandy,CarloD'Este,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,GinaBariKolata,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E.J.W.Barber,W. W. Norton &amp; Company


In [147]:
df_ratings = df_ratings.rename(columns={'Book-Rating': 'Rating'})
df_ratings.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Завдання №3

* Створіть користувача, якому поставте у відповідність 5 довільних кних з переліку та рейтингом (0-10)
* Створіть рекомендований список з 10 книг для користувача, спираючись на автора та видавництво.


In [148]:
user_preferences = {
    'ISBN': ['0004722124', '8511839102', '052165615X', '0521795028', '0515107662'],
    'Rating': [8, 7, 6, 9, 5]
}

inputBooks = pd.DataFrame(user_preferences)
inputBooks

,ISBN,Rating
0,0004722124,8
1,8511839102,7
2,052165615X,6
3,0521795028,9
4,0515107662,5


In [149]:
userSubset = df_ratings[df_ratings['ISBN'].isin(inputBooks['ISBN'].tolist())]
userSubset.head()

,User-ID,ISBN,Rating
3,276729,052165615X,3
4,276729,0521795028,6
13453,1733,0515107662,0
68983,14422,0515107662,8
79618,16795,0515107662,0


In [150]:
userSubsetGroup = userSubset.groupby(['User-ID'])
userSubsetGroup.get_group(276729)

,User-ID,ISBN,Rating
3,276729,052165615X,3
4,276729,0521795028,6


In [151]:
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

<ipython-input-151-b5a0b5f5c9ff>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)


In [152]:
userSubsetGroup[0:3]

[(276729,
     User-ID        ISBN  Rating
  3   276729  052165615X       3
  4   276729  0521795028       6),
 (1733,
         User-ID        ISBN  Rating
  13453     1733  0515107662       0),
 (14422,
         User-ID        ISBN  Rating
  68983    14422  0515107662       8)]

In [153]:
userSubsetGroup = userSubsetGroup[0:100]

In [154]:
#Зберігаємо кореляцію Пірсона у словнику, де ключем є ідентифікатор користувача, а значенням - коефіцієнт
pearsonCorrelationDict = {}

#Для кожної групи користувачів у нашій підмножині
for name, group in userSubsetGroup:
    #Почнемо з сортування вхідних даних та поточної групи користувачів, щоб не переплутати значення
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')
    #Отримаємо N для формули
    nRatings = len(group)
    #Отримаємо оцінки на фільми, які мають спільні риси
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]
    #Збережемо їх у тимчасовій буферній змінній у форматі списку, щоб полегшити майбутні обчислення
    tempRatingList = temp_df['Rating'].tolist()
    #Виведемо оцінки поточної групи користувачів у вигляді списку
    tempGroupList = group['Rating'].tolist()
    #Обчислимо кореляцію Пірсона між двома користувачами, так званими, x та y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Якщо знаменник відмінний від нуля, то ділимо, інакше кореляція 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [156]:
pearsonCorrelationDict.items()

dict_items([(276729, 0.8588285932682869), (1733, 0.06873944968292545), (14422, 0.5357640933905465), (16795, -0.5006963352348883), (26544, -0.8108302163702745), (28076, -0.011599428746363882), (35859, -0.6000794264433851), (45064, -0.8278137904489826), (48494, 0.13860358279576612), (69042, -0.7691806121141458), (76352, 0.6237833835692346), (77940, -0.5660339583473721), (78553, 0.6248964080994224), (78783, 0.2251709877707655), (82164, -0.6948081268374118), (101209, -0.2086821961250822), (110912, 0.11589754016879716), (112559, -0.013943912285765192), (121594, -0.20911854631349325), (125779, 0.5239017687618714), (129358, -0.9242521454438077), (138441, 0.32395433212337554), (144531, -0.3104448657921093), (145451, 0.7528058082467111), (153662, -0.9626544457362882), (154469, -0.6030823030159718), (157116, 0.6141861068150232), (160878, 0.5102404371526978), (170845, 0.3038048463604739), (172030, 0.34220319708406666), (175003, 0.7148362454028483), (180348, 0.49384074072061335), (182085, 0.262752

In [157]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,User-ID
0,0.858829,276729
1,0.068739,1733
2,0.535764,14422
3,-0.500696,16795
4,-0.810830,26544


In [158]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,User-ID
40,0.912038,239535
0,0.858829,276729
45,0.852880,265889
33,0.798963,187239
23,0.752806,145451


In [159]:
topUsersRating=topUsers.merge(df_ratings, left_on='User-ID', right_on='User-ID', how='inner')
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Rating
0,0.912038,239535,006018311X,0
1,0.912038,239535,0060972009,0
2,0.912038,239535,0060975008,0
3,0.912038,239535,0060977477,0
4,0.912038,239535,0060987103,10


In [160]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Rating,weightedRating
0,0.912038,239535,006018311X,0,0.000000
1,0.912038,239535,0060972009,0,0.000000
2,0.912038,239535,0060975008,0,0.000000
3,0.912038,239535,0060977477,0,0.000000
4,0.912038,239535,0060987103,10,9.120381


In [161]:
#Знайдемо суму до topUsers після групування за userId
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
ISBN,,
000123207X,-0.161160,0.0
0001232088,-0.161160,0.0
0001931377,-0.161160,0.0
0001981625,0.623783,0.0
0002220296,-0.769181,0.0


In [162]:
#Створюємо порожній фрейм даних
recommendation_df = pd.DataFrame()
#Тепер беремо середньозважене значення
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,ISBN
ISBN,,
000123207X,-0.0,000123207X
0001232088,-0.0,0001232088
0001931377,-0.0,0001931377
0001981625,0.0,0001981625
0002220296,-0.0,0002220296


In [163]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,ISBN
ISBN,,
0312982623,5195.574029,0312982623
0373226411,3259.609821,0373226411
0373218435,2612.434430,0373218435
0061031410,2223.116322,0061031410
0553299468,821.583053,0553299468
067103412X,667.239679,067103412X
0778320111,576.603016,0778320111
0671778684,405.301649,0671778684
0316168815,323.790164,0316168815


In [164]:
#Підсумкова таблиця рекомендацій
df_books_selected.loc[df_books_selected['ISBN'].isin(recommendation_df.head(10)['ISBN'].tolist())]

,ISBN,Book-Title,Book-Author,Publisher
474,0316168815,The Lovely Bones,AliceSebold,Back Bay Books
6404,0061031410,Courting Trouble,LisaScottoline,HarperTorch
21410,0553299468,Midnight Warrior,IRISJOHANSEN,Bantam
30765,0778320111,Suburban Renewal,PamelaMorsi,Mira Books
31438,0373226411,Lassiter'S Law (43 Light Street/Mine To Keep) ...,RebeccaYork,Harlequin
47972,0312982623,Deadly Affairs (A Francesca Cahill Romance),BrendaJoyce,St. Martin's Paperbacks
